In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


# Getting Started Competition

In [2]:
import torch
import numpy as np
import pandas as pd
import spacy
from torch.utils.data import Dataset
from transformers import EvalPrediction, Trainer, TrainingArguments, DistilBertTokenizerFast
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Load the tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

## Notebook Setup

The first thing we need to do is setup our notebook and clean the data.

1. So we load the data into pandas
2. Check the first few rows of our data
3. Then we drop rows with n/a data

In [3]:
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = self.preprocess_text(texts)
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # Check if we have labels
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(label, dtype=torch.long),
            }
        else:  # Return only inputs for test data
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
            }


    def preprocess_text(self, texts):
        preprocessed_texts = []
        for text in texts:
            # Apply SpaCy pipeline on the text
            doc = nlp(text)
            # Lemmatize the text and join the words back into a single string
            lemma_text = " ".join([token.lemma_ for token in doc])
            preprocessed_texts.append(lemma_text)
        return preprocessed_texts


def compute_metrics(eval_pred: EvalPrediction):
    labels = eval_pred.label_ids
    preds = np.argmax(eval_pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



## Load The Data

In [4]:
# Handle missing values by filling with a placeholder value
train_df = train_df.fillna('None')
test_df = test_df.fillna('None')

## Augment the Code

In [5]:
# Combine train and test data One-hot encode the 'keyword' field
combined_df = pd.concat([train_df, test_df])
combined_df = pd.get_dummies(combined_df, columns=['keyword'])

# Split back into train and test data
train_df = combined_df[:len(train_df)]
test_df = combined_df[len(train_df):]

# Extract the texts from the training data
train_texts = train_df['text'].tolist()

# Perform data augmentation
import nlpaug.augmenter.word as naw

aug = naw.SynonymAug(aug_src='wordnet')
augmented_texts = [aug.augment(text) for text in train_texts]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 train_texts = train_df['text'].tolist()                                                     │
│   11                                                                                             │
│   12 # Perform data augmentation                                                                 │
│ ❱ 13 import nlpaug.augmenter.word as naw                                                         │
│   14                                                                                             │
│   15 aug = naw.SynonymAug(aug_src='wordnet')                                                     │
│   16 augmented_texts = [aug.augment(text) for text in train_texts]                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'nlpaug'

In [ ]:
train_df = train_df.copy()
test_df = test_df.copy()

train_df.loc[:, 'input'] = train_df.apply(lambda row: f"{row['text']}", axis=1)
test_df.loc[:, 'input'] = test_df.apply(lambda row: f"{row['text']}", axis=1)

# Extract 'input' as input and 'target' as labels from the train data
train_texts = train_df['input'].tolist()
train_labels = train_df['target'].tolist()

# For the test data, we only need the 'input' column
test_texts = test_df['input'].tolist()


# Split the train data into train and validation subsets
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2)

# Print the sizes of the training and validation subsets
print(f"Training set size: {len(train_texts)}")
print(f"Validation set size: {len(val_texts)}")

In [ ]:
# Combine train and test data
combined_df = pd.concat([train_df, test_df])

# One-hot encode the 'keyword' field
combined_df = pd.get_dummies(combined_df, columns=['keyword'])

# Split back into train and test data
train_df = combined_df[:len(train_df)]
test_df = combined_df[len(train_df):]



## Initialize the datasets

In [ ]:
train_dataset = TweetDataset(train_texts, train_labels, tokenizer)
val_dataset = TweetDataset(val_texts, val_labels, tokenizer)
test_dataset = TweetDataset(test_texts, None, tokenizer)  # No labels for the test data

## Load bert

In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2, hidden_dropout_prob=0.3)

### Split the data into training and validation sets

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Define the training arguments
training_args = TrainingArguments(
    output_dir='../models/results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    evaluation_strategy='epoch',     # evaluation is performed at the end of each epoch
    fp16=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,      # function to compute metrics
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=3)
    ],  # Stop if validation loss doesn't improve for 3 evaluations
)

## Train The Model

In [ ]:
# Train the model
trainer.train()

### Plot confusion matrix

In [ ]:
# Get predictions
predictions = trainer.predict(train_dataset)

# The predictions are in a tuple with the loss, we only need the actual predictions
train_preds = np.argmax(predictions.predictions, axis=1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Compute the confusion matrix
conf_matrix = confusion_matrix(train_labels, train_preds, labels=[0, 1])

# Create the plot
myplt = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0, 1])
myplt.plot()

# Show the plot
plt.show()


In [ ]:
# Get predictions
predictions = trainer.predict(val_dataset)

# The predictions are in a tuple with the loss, we only need the actual predictions
val_preds = np.argmax(predictions.predictions, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(val_labels, val_preds, labels=[0, 1])

# Create the plot
myplt = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0, 1])
myplt.plot()

# Show the plot
plt.show()


## More Visuals

### 1. **Distribution of Predictions**

This can be useful to see if your model is biased towards predicting a certain class.

In [ ]:
import seaborn as sns

sns.countplot(x=test_preds)
plt.title('Distribution of Predictions')
plt.show()

### 2. **Distribution of Actual Labels**

This can be useful to compare with the distribution of predictions.

In [ ]:
sns.countplot(x=train_labels)
plt.title('Distribution of Actual Labels')
plt.show()

### 3. **ROC Curve**

This is a common way to visualize the performance of a binary classifier. \
The closer the curve is to the top-left corner, the better the classifier.

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(val_labels, val_preds)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

### 4. **Loss Curve**

If you stored the loss at each epoch during training, you can plot it to see how the model improved over time.

In [ ]:
"""
plt.plot(training_loss)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()
"""

### Submission Output to CSV file


In [ ]:
# Save the model
trainer.save_model("/kaggle/working/models")

### make test predictions

In [ ]:
# Making predictions on test data
test_predictions = trainer.predict(test_dataset)

# We take the output class with the highest probability
test_preds = np.argmax(test_predictions.predictions, axis=1)

# Prepare a DataFrame with test IDs and predictions
submission_df = pd.DataFrame({
    'id': test_df['id'],  # Assuming that 'test_df' is the DataFrame with your test data
    'target': test_preds
})

### 1. **Confusion Matrix**

This is a table that is often used to describe the performance of a classification model (or "classifier") on a set of test data for which the true values are known.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming true_labels and pred_labels are your data
true_labels = [...]
pred_labels = [...]

conf_mat = confusion_matrix(true_labels, pred_labels)
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

### 2. **Classification Report**

This is another way to evaluate the classification model, it displays the precision, recall, F1, and support scores for the model.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(true_labels, pred_labels, target_names=['Negative', 'Positive']))


### 3. **ROC Curve**

The Receiver Operating Characteristic curve is a graphical plot that illustrates the diagnostic ability of a binary classifier system as its discrimination threshold is varied.


In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(true_labels, pred_labels)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

## Submission

Save the DataFrame into a CSV file

In [ ]:
submission_df.to_csv('submission.csv', index=False)